# TRENDS DIFFERENT SIZE SET1


### ENERGY ERROR

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from pyfonts import load_font
from matplotlib.font_manager import FontProperties

# load font
helvetica_bold = load_font("/home/seraf/Downloads/helvetica-255/Helvetica-Bold.ttf")
helv_bold_prop = FontProperties(fname="/home/seraf/Downloads/helvetica-255/Helvetica-Bold.ttf", size=35)

helvetica_strange = load_font("/home/seraf/Downloads/helvetica-255/helvetica-light-587ebe5a59211.ttf")
helv_strange_prop = FontProperties(fname="/home/seraf/Downloads/helvetica-255/helvetica-light-587ebe5a59211.ttf", size=20)


mother_dir = './HvsINIT/'

title = 'SET1 Prediction Performance'
fig, ax = plt.subplots(1, 1, figsize=(15, 10))
fig.subplots_adjust(hspace=0.0)
ax.set_title(title, fontproperties=helv_bold_prop, pad=10)
max_x_ticks = np.array([])

complist = [1]
for comp in complist:
    dir_name = f'{comp}'
    df_tot = pd.read_csv(mother_dir + f'{dir_name}/TotalStaticsBefore.csv', header=None, na_filter=False, index_col = 0)
    tot = (df_tot.to_numpy()).T * 1000

    x = np.arange(1, len(tot[0])+1) 

    ax.plot(x, tot[0] , marker = 'o', markeredgecolor='white', markersize=10, label=f'A$_{comp}$B')
    max_x_ticks = np.append(max_x_ticks, len(tot[0]))

ax.set_ylabel(r'$\Delta$H $_{USPEX-TEMP}$ [meV/atom]', fontsize=25, font=helvetica_strange, labelpad=10)
step = 20
# y_ticks  = np.arange(0,250,step, dtype=float)
# ax.set_yticks(y_ticks)
# ax.set_yticklabels([f'{i:.0f}' for i in y_ticks])
ax.set_ylim(0,250)
ax.yaxis.set_major_locator(plt.MultipleLocator(20))


ax.set_xlabel('Size SET1', fontsize=25, font=helvetica_strange, labelpad=10)
xticks = np.arange(0, max(max_x_ticks)+1, 10)
ax.set_xticks(xticks)
ax.set_xlim(1,max(max_x_ticks)+1)
ax.set_xticklabels([f'{i:.0f}' for i in xticks])

ax.tick_params(size=7, labelsize=15)
for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontproperties(helv_strange_prop)

ax.grid(True, ls=':')
ax.legend(loc='upper center', prop=helv_strange_prop, ncol=5)

plt.show()
fig.savefig('FIG/A1B.png')


### SIZE FINAL SET

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

mother_dir = './HvsINIT/'

complist = [1, 2, 3, 4]

title = f'SIZE FINAL SET with threshold 0.9'
fig, ax = plt.subplots(1, 1, figsize=(15, 7))
fig.subplots_adjust(hspace=0.0)
fig.suptitle(title, fontsize=20, y=0.95, fontweight='bold')

max_x_ticks = np.array([])

for comp in complist:
    dir_name = f'{comp}'
    df_numtemp = pd.read_csv(mother_dir + f'{dir_name}/NumberTempRedu.csv', header=None, na_filter = False, index_col = 0)
    numtemp = (df_numtemp.to_numpy()).T
    x = np.arange(1, len(numtemp[0])+1)

    ax.plot(x, numtemp[0], marker = 'o', markeredgecolor='white', markersize=6, label=f'A$_{comp}$B')
    max_x_ticks = np.append(max_x_ticks, len(numtemp[0]))

ax.set_ylabel('Size final set', fontsize=22)
ax.yaxis.set_label_coords(-0.07, 0.5)

step = 2
y_ticks  = np.arange(0,30,step, dtype=float)
y_ticks = np.round(y_ticks, decimals=0)
ax.set_yticks(y_ticks)
ax.set_yticklabels([f'{i:.0f}' for i in y_ticks], fontsize=18)
ax.set_ylim(0,30)

ax.set_xlabel('Size initial set', fontsize=22)
ax.xaxis.set_label_coords(0.5, -0.10)

xticks = np.arange(1, max(max_x_ticks)+1, 5)
xticks = np.round(xticks, decimals=0) 
ax.set_xticks(xticks)
ax.set_xlim(0,max(max_x_ticks)+1)
ax.set_xticklabels([f'{i:.0f}' for i in xticks], fontsize=15, rotation=45)

ax.grid(True, ls=':')
ax.legend(loc='lower right', fontsize=15)

plt.show()
fig.savefig('FIG/INIT_Ntemp.png', bbox_inches="tight")


### STD ENERGY

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

mother_dir = './HvsINIT/'

title = f'$\Delta$H w.r.t. USPEX GS with INITIAL SET'
fig, ax = plt.subplots(1, 1, figsize=(15, 7))
fig.subplots_adjust(hspace=0.0)
fig.suptitle(title, fontsize=20, y=0.95, fontweight='bold')
max_x_ticks = np.array([])

complist = [1,2,3,4]
for comp in complist:
    dir_name = f'{comp}'
    df_tot = pd.read_csv(mother_dir + f'{dir_name}/TotalStaticsBefore.csv', header=None, na_filter=False, index_col = 0)
    tot = (df_tot.to_numpy()).T

    x = np.arange(1, len(tot[1])+1) 

    ax.plot(x, tot[1] , marker = 'o', markeredgecolor='white', markersize=6, label=f'A$_{comp}$B')
    max_x_ticks = np.append(max_x_ticks, len(tot[1]))


ax.set_ylabel(r'$\Delta$H [eV/atom]', fontsize=22)
ax.yaxis.set_label_coords(-0.07, 0.5)

step = 0.005
y_ticks  = np.arange(0.0,0.1,step, dtype=float)
y_ticks = np.round(y_ticks, decimals=2)
ax.set_yticks(y_ticks)
ax.set_yticklabels([f'{i:.3f}' for i in y_ticks], fontsize=18)
ax.set_ylim(0,0.1)

ax.set_xlabel('Size initial set', fontsize=22)
ax.xaxis.set_label_coords(0.5, -0.10)

xticks = np.arange(1, max(max_x_ticks)+1, 5)
xticks = np.round(xticks, decimals=0) 
ax.set_xticks(xticks)
ax.set_xlim(0,max(max_x_ticks)+1)
ax.set_xticklabels([f'{i:.0f}' for i in xticks], fontsize=15, rotation=45)

ax.grid(True, ls=':')
ax.legend(loc='upper right', fontsize=15)

plt.show()
fig.savefig('FIG/INIT_std.png', bbox_inches="tight")


# TEMPLATE ANALYSIS

### BEST TEMPLATES

In [ ]:
import json
import matplotlib.pyplot as plt

color_list = ['blue', 'red', 'green', 'purple']
comp_list = [1,2,3,4]

for idx, comp in enumerate(comp_list):
    if comp == 1:
        num_pairs = 105
    else:
        num_pairs = 210
    with open(f'ENTHALPY/A{comp}B.json', 'r') as f:
        data = json.load(f)

    with open(f'ENTHALPY/EntGS.json', 'r') as f:
        gs_dict = json.load(f)

    title = r'$\Delta$H for each template averaged over all pairs'
    fig, ax = plt.subplots(1, 1, figsize=(25, 7))

    temp_dict = {}

    count = 0
    for pair in data.keys():
        count +=1
        for temp in data[pair].keys():
            if temp not in temp_dict.keys():
                temp_dict[temp] = (data[pair][temp] - gs_dict[f'{comp}'][pair])/num_pairs
            else:
                temp_dict[temp] += (data[pair][temp] - gs_dict[f'{comp}'][pair])/num_pairs
    
    # Ordina i template in base ai valori delle entalpie
    sorted_items = sorted(temp_dict.items(), key=lambda x: x[1])  
    sorted_keys, sorted_values = zip(*sorted_items)  # Divide in due liste ordinate


    fig.suptitle(title, fontsize=20, y=0.95, fontweight='bold')
    ax.bar(sorted_keys, sorted_values, color=color_list[idx], alpha=0.7)
    ax.set_ylabel(r'$\Delta$H [eV/atom]', fontsize=22, labelpad=10)

    ax.set_xlim(-1, len(temp_dict.keys()))
    ax.set_xlabel('Template', fontsize=22, labelpad=10)
    ax.set_xticks(range(len(sorted_keys)))
    fontsize = 12
    if comp == 2:
        fontsize = 10
    ax.set_xticklabels([f'{i.split("_")[0]} {i.split("_")[1]}' for i in sorted_keys], fontsize=fontsize, rotation=90)


    ax.grid(True, ls=':', axis='y')





### ADAPTABILITY OVER TEMPLATES

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, leaves_list

# Carica i dati
with open('ENTHALPY/EntGS.json', 'r') as f:
    gs_dict = json.load(f)

elements = ['H', 'Li', 'O', 'Na', 'N', 'Mg', 'Si', 'S', 'Be', 'B', 'Al', 'C', 'F', 'P', 'Cl']

color_list = ['blue', 'red', 'green', 'purple']

for idx, comp in enumerate(gs_dict.keys()):
    if comp in ['6']:
        continue

    pair_count_dict = {}
    with open(f'ENTHALPY/A{comp}B.json', 'r') as f:
        data_ent = json.load(f)
    
    for pair in data_ent.keys():
        pair_count_dict[pair] = 0
        for temp in data_ent[pair].keys():
            if data_ent[pair][temp] - gs_dict[comp][pair] < 0.1:
                pair_count_dict[pair] += 1

    sorted_items = sorted(pair_count_dict.items(), key=lambda x: x[1])  
    sorted_keys, sorted_values = zip(*sorted_items)  # Divide in due liste ordinate

    fig, ax = plt.subplots(1, 1, figsize=(10, 30))
    title = r'Number of templates with $\Delta$H < 0.1 eV/atom' + f' A$_{comp}$B'
    fig.suptitle(title, fontsize=20, y=0.9, fontweight='bold')
    
    ax.barh(sorted_keys, sorted_values, color=color_list[idx], alpha=0.7)
    ax.set_xlabel(r'Number of templates', fontsize=22, labelpad=10, family='serif')

    ax.set_ylim(-1, len(pair_count_dict.keys()))
    ax.set_ylabel('Pair', fontsize=22, labelpad=10, family='serif')
    ax.set_yticks(range(len(sorted_keys)))
    fontsize = 12
    if comp != '1':
        fontsize = 8
    ax.set_yticklabels([f'{i}' for i in sorted_keys], fontsize=fontsize)


    ax.grid(True, ls=':', axis='y')

    

    plt.show()


# TRENDS WITHOUT CRITICAL PAIRS

### PRODUCTION OF DATA

In [ ]:
from template_csp import managetemp_withdict as mt
import json
import os
import numpy as np

from tqdm import tqdm

test_elements = ['Be', 'B', 'N', 'Mg', 'O', 'Li', 'C', 'Na', 'Si', 'S', 'Cl', 'F', 'P', 'H', 'Al']

hyperparameters = { 
    "weight_occurrence" : 1,
    "weight_sg" : 0.001,
    "weight_formation_entalphy" : 1, 
    "comp" : 1,
    "lev_red" : 0.9,
    "n_pairs" : 105,

    "n_final_templates" : 1
}


complist = [1, 2, 3, 4] 
initlist = [86, 156, 92, 79] # 86 
mother_dir = './HvsINIT/'
path_to_critical_pairs = 'CriticalPairs.json'

for comp in complist:
    dir_comp = mother_dir + f'{comp}/'
    means = np.zeros(initlist[complist.index(comp)])
    stds = np.zeros(initlist[complist.index(comp)])

    for size_initial in tqdm(range(1, initlist[complist.index(comp)]+1)):
        dir_init = dir_comp + f'{size_initial}/'
        errors_bef = np.zeros(20)

        
        for step in range(0,20):
            file_restart = dir_init + f'TemplateSet_{step}'

            initial_set = mt.TemplateSet(test_elements=test_elements, hyperparameters=hyperparameters, restart_file=file_restart, comp = comp)
            errors_bef[step] = initial_set.err_before(path_to_critical_pairs)

        means[size_initial-1] = np.mean(errors_bef)
        stds[size_initial-1] = np.std(errors_bef)

        if size_initial == 1:
            with open(dir_comp + 'BeforeNoCritical0.csv', 'w') as f:
                f.write(f'{size_initial-1},{means[size_initial-1]},{stds[size_initial-1]}\n')
        else:
            with open(dir_comp + 'BeforeNoCritical0.csv', 'a') as f:
                f.write(f'{size_initial-1},{means[size_initial-1]},{stds[size_initial-1]}\n')
        

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

mother_dir = './HvsINIT/'

crit_pairs = {}
with open('CriticalPairs.json', 'r') as f:
    crit_pairs = json.load(f)


title = r'$\Delta$H w.r.t. USPEX GS with INITIAL SET'
fig, ax = plt.subplots(1, 1, figsize=(15, 7))
fig.subplots_adjust(hspace=0.0)
fig.suptitle(title, fontsize=25, y=0.95, fontweight='bold')
max_x_ticks = np.array([])

complist = [1,2,3,4]
for comp in complist:
    dir_name = f'{comp}'
    df_tot = pd.read_csv(mother_dir + f'{dir_name}/BeforeNoCritical0.csv', header=None, na_filter=False, index_col = 0)
    tot = (df_tot.to_numpy()).T * 1000

    x = np.arange(1, len(tot[0])+1) 

    ax.plot(x, tot[0] , marker = 'o', markeredgecolor='white', markersize=8, label=f'A$_{comp}$B - {len(crit_pairs[f"{comp}"])} excluded')
    max_x_ticks = np.append(max_x_ticks, len(tot[0]))


ax.set_ylabel(r'$\Delta$H [meV/atom]', fontsize=22, labelpad=10, family='serif')
step = 20
y_ticks  = np.arange(0,150,step, dtype=float)
ax.set_yticks(y_ticks)
ax.set_yticklabels([f'{i:.0f}' for i in y_ticks], fontsize=18)
ax.set_ylim(0,150)

ax.set_xlabel('SIZE INITIAL SET', fontsize=22, labelpad=10, family='serif')
xticks = np.arange(0, max(max_x_ticks)+1, 10)
ax.set_xticks(xticks)
ax.set_xlim(1,max(max_x_ticks)+1)
ax.set_xticklabels([f'{i:.0f}' for i in xticks], fontsize=18)

ax.grid(True, ls=':')
ax.legend(loc='upper right', fontsize=15)

plt.show()
fig.savefig('FIG/INIT_H_noCritical.png', bbox_inches="tight")


# PROVE

In [ ]:
from template_csp import manage_alltemp as mte 
import json
from tqdm import tqdm

test_elements = ['Be', 'B', 'N', 'Mg', 'O', 'Li', 'C', 'Na', 'Si', 'S', 'Cl', 'F', 'P', 'H', 'Al']

hyperparameters = { 
    "weight_occurrence" : 1,
    "weight_sg" : 0.001,
    "weight_formation_entalphy" : 1, 
    "comp" : 1,
    "lev_red" : 0.9,
    "n_pairs" : 105,

    "n_template" : 1,

    "n_final_templates" : 1,
    "job_id" : 0
    #"critical_pairs" : './CriticalPairs.json'
}

data = {}
data1 = {}

complist = [1,2,3,4,6]
temp_list = [[20,30,40], [40,50,60], [30,40,50], [30,40,50], [20,30,40]] 
temp_final = [10, 20, 15, 15, 10]
#total_pairs_list = [95, 180, 175, 171, 210]
total_pairs_list = None
n_set = 20
n_try_pairs = 30

crit_pairs = './CriticalPairs.json'

for comp, temp_init_list in tqdm(zip(complist, temp_list)):
    
    if comp == 6:
        continue

    hyperparameters["comp"] = comp
    
    if total_pairs_list:
        total_pairs = total_pairs_list[complist.index(comp)]
    else:
        if comp == 1:
            total_pairs = 105
        else:
            total_pairs = 210

    data[comp] = {'ErrBef': {}, 'ErrAft': {}}
    data1[comp] = {}

    hyperparameters['n_final_templates'] = temp_final[complist.index(comp)]

    for idx_temp, temp_init in enumerate(temp_init_list):

        data[comp]['ErrBef'][temp_init] = 0
        data[comp]['ErrAft'][temp_init] = {}
        data1[comp][temp_init] = {}    

        for id_set in range(n_set):

            init_set = mte.InitialSet(test_elements, hyperparameters, 'InitialSet.txt')
            # init_set.recap()
            
            data[comp]['ErrBef'][temp_init] += init_set.difference_from_uspex()/n_set
            
            for idx_npair, npair in enumerate(range(1, total_pairs, 5)):

                if npair not in data[comp]['ErrAft'][temp_init].keys():
                    data[comp]['ErrAft'][temp_init][npair] = 0
                    data1[comp][temp_init][npair] = 0

                hyperparameters["n_pairs"] = npair
                
                for try_pairs in range(n_try_pairs):
                    final_set = mte.FinalSet(init_set, test_elements, hyperparameters, 'FinalSet.txt')
                    
                    data[comp]['ErrAft'][temp_init][npair] += final_set.difference_from_uspex()/(n_set*n_try_pairs)
                    data1[comp][temp_init][npair] += float(final_set.count_crit_in_validation)/(n_set*n_try_pairs)

                    # final_set.recap()
        
            
            npair = total_pairs
            hyperparameters["n_pairs"] = npair

            final_set = mte.generate_final_set(init_set, hyperparameters=hyperparameters, test_elements=test_elements, file_crit_pairs = crit_pairs)
            if npair not in data[comp]['ErrAft'][temp_init].keys():
                data[comp]['ErrAft'][temp_init][npair] = 0
                data1[comp][temp_init][npair] = 0
            
            data1[comp][temp_init][npair] += float(final_set.count_crit_in_validation)/n_set
            data[comp]['ErrAft'][temp_init][npair] += final_set.difference_from_uspex()/n_set

            with open('./NumberOfPairs_Continuum.json', 'w') as f:
                json.dump(data, f, indent=4)
                
            with open('./UsedCritPairs.json', 'w') as f:
                json.dump(data1, f, indent=4) 
